# 6 - Diseño experimental y particionado de datos


## Preparación del entorno


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer, make_classification
from sklearn.model_selection import (train_test_split, StratifiedKFold, RepeatedStratifiedKFold,
                                      GroupKFold, LeaveOneGroupOut, cross_val_score,
                                      GridSearchCV)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

import json, os, sys, platform, random, time

In [ ]:
# Reproducibilidad
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

**Dataset:** `load_breast_cancer` (*clasificación binaria)*

In [ ]:
data = load_breast_cancer(as_frame=True)

df = data.frame.copy()
X = df.drop(columns=['target'])
y = df['target']

display(X.head(3))
print(f"Dimensiones: {X.shape}\n\nDistribución del {y.value_counts(normalize=True)}")

## 1. Ejemplo de pipeline estándar
Un pipeline mínimo para clasificación binaria con datos numéricos:

1) `SimpleImputer(strategy='median')` → 2) `StandardScaler()` → 3) `LogisticRegression()`

In [ ]:
final_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=500, random_state=SEED))
])
final_pipe

## 2. Conjuntos Train / Validación / Test: roles y reglas
Creamos particiones **estratificadas** y guardamos los índices para reproducibilidad.


In [ ]:
# Holdout estratificado: primero Train+Valid vs Test
X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
# Ahora Train vs Valid
X_train, X_valid, y_train, y_valid = train_test_split(X_tv, y_tv, test_size=0.25, stratify=y_tv, random_state=SEED)  # 0.25 de 0.8 -> 0.2

# Guardamos índices para auditoría
os.makedirs('artifacts', exist_ok=True)
splits = {
    'seed': SEED,
    'train_idx': X_train.index.to_list(),
    'valid_idx': X_valid.index.to_list(),
    'test_idx': X_test.index.to_list()
}
with open('artifacts/holdout_indices.json', 'w') as f:
    json.dump(splits, f)

print(f"Número de registros:\n- Train: {len(X_train)}\n- Validación: {len(X_valid)}\n- Test: {len(X_test)}")


## 3. Holdout simple (baseline)
Entrenamos un **pipeline sin fugas** y evaluamos en **Validación** y **Test**.


In [ ]:
start = time.time()
final_pipe.fit(X_train, y_train)
print(f"Duración: {round(time.time() - start, 2)}s\n")

val_auc = roc_auc_score(y_valid, final_pipe.predict_proba(X_valid)[:,1])
test_auc = roc_auc_score(y_test, final_pipe.predict_proba(X_test)[:,1])

print(f"AUC conjunto Validación: {round(val_auc, 4)}")
print(f"AUC conjunto Test: {round(test_auc, 4)}")

## 4. k-fold Cross-Validation (estratificada)
Estimamos AUC promediando sobre **K=5** folds estratificados.


In [ ]:
cv5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

start = time.time()
scores_cv5 = cross_val_score(final_pipe, X, y, cv=cv5, scoring='roc_auc')
print(f"Duración: {round(time.time() - start, 2)}s\n")

print(f"Media AUC: {round(scores_cv5.mean(), 4)}\nDesv. estándar AUC: {round(scores_cv5.std(), 4)}")

## 5. Repeated k-fold (5x3)
Reducimos varianza repitiendo el k-fold con **diferentes semillas internas**.


In [ ]:
rcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=SEED)

start = time.time()
scores_rcv = cross_val_score(final_pipe, X, y, cv=rcv, scoring='roc_auc')
print(f"Duración: {round(time.time() - start, 2)}s\n")

print(f"Media AUC: {round(scores_rcv.mean(), 4)}\nDesv. estándar AUC: {round(scores_rcv.std(), 4)}")

## 6. Demostración de data leakage (incorrecto vs correcto)
Comparamos un **protocolo incorrecto** (escalado previo con toda la data) vs. el **protocolo correcto** (escalado dentro del `Pipeline`).


In [ ]:
# INCORRECTO: transformamos antes del CV (posible fuga)
X_scaled_wrong = StandardScaler().fit_transform(X)
auc_wrong = cross_val_score(LogisticRegression(max_iter=500, random_state=SEED),
                            X_scaled_wrong, y, cv=cv5, scoring='roc_auc')

# CORRECTO: escalado dentro del Pipeline
auc_right = cross_val_score(final_pipe, X, y, cv=cv5, scoring='roc_auc')

print(f"Media AUC (fuga datos): {round(auc_wrong.mean(), 4)}")
print(f"Media AUC (correcto): {round(auc_right.mean(), 4)}")

## 7. Nested Cross-Validation (tuning honesto)
Separamos **búsqueda de hiperparámetros** (inner CV) de **estimación** (outer CV).


In [ ]:
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)

# Optimización de hiperparámetros: veremos el detalle en sesiones posteriores
param_grid = {'clf__C': [0.1, 1, 10, 100],
              'clf__penalty': ['l2'],
              'clf__solver': ['lbfgs', 'liblinear']}
search = GridSearchCV(final_pipe, param_grid=param_grid, scoring='roc_auc', cv=inner_cv)

start = time.time()
nested_scores = cross_val_score(search, X, y, cv=outer_cv, scoring='roc_auc')
print(f"Duración: {round(time.time() - start, 2)}s\n")

print(f"Media AUC: {round(nested_scores.mean(), 4)}\nDesv. estándar AUC: {round(nested_scores.std(), 4)}")

## 8. Validación con grupos: GroupKFold y LOGO
Simulamos **entidades** para demostrar la validación por grupos.


In [ ]:
# Creación de un dataset sintético con 80 grupos de tamaño 10
Xg, yg = make_classification(n_samples=800, n_features=15, n_informative=8, n_redundant=2,
                              n_clusters_per_class=2, weights=[0.6, 0.4], flip_y=0.02,
                              class_sep=1.0, random_state=SEED)
groups = np.repeat(np.arange(80), 10)

gkf = GroupKFold(n_splits=5)

start = time.time()
scores_gkf = cross_val_score(final_pipe, Xg, yg, groups=groups, cv=gkf, scoring='roc_auc')
print(f"Duración: {round(time.time() - start, 2)}s\n")

print(f"Media AUC: {round(scores_gkf.mean(), 4)}\nDesv. estándar AUC: {round(scores_gkf.std(), 4)}")

In [ ]:
logo = LeaveOneGroupOut()

start = time.time()
scores_logo = cross_val_score(final_pipe, Xg, yg, groups=groups, cv=logo, scoring='roc_auc')
print(f"Duración: {round(time.time() - start, 2)}s\n")

print(f"Nº de iteraciones = Nº de grupos -> {len(scores_logo)}\n")
print(f"Media AUC: {round(scores_logo.mean(), 4)}\nDesv. estándar AUC: {round(scores_logo.std(), 4)}")

## Ejercicio
Implementa un **flujo completo sin fugas** siguiendo estas indicaciones:

1. Crea y justifica un **holdout** estratificado (Train/Valid/Test). Reporta AUC en Valid y Test.
2. Compara **k-fold (K=5)** y **repeated k-fold (5x3)** con el mismo pipeline. Reporta media y desviación de AUC.
3. Realiza **Nested CV** (outer=5, inner=3) con grid de `C` para `LogisticRegression`. Reporta el AUC medio.
4. Simula **grupos** (o usa un dataset con entidades) y evalúa **GroupKFold (K=5)** y **LOGO**. Compara estabilidad.
5. Demuestra un caso de **data leakage** y su corrección con `Pipeline`.
6. Entrega los **índices** de los splits usados (JSON en `artifacts/`).
